In [9]:
%load_ext sql


In [11]:
%%sql sqlite://
SELECT * FROM some_table;

(sqlite3.OperationalError) no such table: some_table
[SQL: SELECT * FROM some_table;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [12]:
import pandas as pd
import sqlite3

# Load your Excel/CSV data
df_orders = pd.read_csv("Retail Order - Retails Order Full Dataset.csv")
df_calendar = pd.read_csv("Calender Table - Calendar Table.csv")

# Connect to SQLite
conn = sqlite3.connect("retail_supply_chain.db")

# Push DataFrame to SQLite
df_orders.to_sql("orders", conn, if_exists="replace", index=False)
df_calendar.to_sql("calendar", conn, if_exists="replace", index=False)


1826

In [14]:
query = """
SELECT 
    category,
    SUM(sales) AS total_sales,
    SUM(profit) AS total_profit
FROM orders
GROUP BY category
ORDER BY total_profit DESC;
"""

df_result = pd.read_sql_query(query, conn)
df_result.head()


,Category,total_sales,total_profit
0,Technology,836088.11,145438.50
1,Office Supplies,719040.62,122500.44
2,Furniture,741999.98,18451.25


### Profitability Focus
- **Technology** generates the highest profit margins (**17.4%**) despite not having the highest sales, suggesting efficient supply chain management (better sourcing, inventory turnover).

### Furniture Challenge
- Lowest profit margin (**2.5%**) indicates high logistics/warehousing costs or markdowns.
- Recommendations: Optimize supplier contracts or rethink distribution strategy.

### Balanced Portfolio
- **Office Supplies** show steady performance (**17%** margin).
- Maintain reliable suppliers and lean inventory for consistent cash flow.

### Action
- Prioritize Technology supply chain resilience.
- Renegotiate Furniture supplier terms.
- Monitor Office Supplies demand patterns.


#### 1. Inventory Turnover Proxy

In [34]:
query = """
SELECT 
    Category,
    ROUND(SUM(Sales) / SUM(Quantity), 2) AS avg_sales_per_unit
FROM orders
GROUP BY Category
ORDER BY avg_sales_per_unit DESC;
"""

pd.read_sql_query(query, conn)


,Category,avg_sales_per_unit
0,Technology,120.51
1,Furniture,92.43
2,Office Supplies,31.40


### Supply Chain Insights

1. **Premium Pricing in Technology**  
   - High average sales per unit (**$120.51**) suggests high-value products.  
   - Ensure reliable suppliers and minimal stockouts.

2. **Furniture Mid-Range**  
   - Average sales per unit at **$92.43** indicates bulk and logistics costs.  
   - Optimize warehousing and shipping.

3. **Office Supplies Low-Cost**  
   - Average sales per unit at **$31.40** requires lean inventory and cost-efficient suppliers.

### Action
- Prioritize Technology supply chain stability.  
- Streamline Furniture logistics.  
- Minimize Office Supplies overhead.


### 2. Top 5 Returned Products by Volume

In [17]:
query = """
SELECT 
    "Product Name",
    COUNT(*) AS return_count
FROM orders
WHERE Returned = 'Yes'
GROUP BY "Product Name"
ORDER BY return_count DESC
LIMIT 5;
"""

pd.read_sql_query(query, conn)


,Product Name,return_count
0,Staple envelope,4
1,KI Adjustable-Height Table,4
2,Xerox 1882,3
3,"Wirebound Service Call Books, 5 1/2"" x 4""",3
4,"Wilson Jones Hanging View Binder, White, 1""",3


### Supply Chain Insights

1. **High-Return Items**  
   - Staples and tables have the most returns (**4 each**), indicating potential quality or shipping issues.

2. **Supplier Review Needed**  
   - Investigate whether defects in products like Xerox and binders arise from manufacturing flaws or handling errors.

### Action
- Conduct supplier audits to identify root causes of returns.  
- Improve packaging to reduce damage during transit.  
- Negotiate return cost clauses with suppliers to manage financial impact.


### 3. Monthly Sales and Profit Trend

In [33]:
query = """
SELECT 
    Category,
    ROUND(SUM(Sales) / SUM(Quantity), 2) AS avg_sales_per_unit
FROM orders
GROUP BY Category
ORDER BY avg_sales_per_unit DESC;

"""

pd.read_sql_query(query, conn)


,Category,avg_sales_per_unit
0,Technology,120.51
1,Furniture,92.43
2,Office Supplies,31.40


### Key Insight

- Technology drives **3.8x higher revenue per unit** than Office Supplies, highlighting its premium value.
- **Priority:** Protect margins by securing critical technology components first.
- **Risk:** Overstocking Furniture, which has mid-range value, ties up capital and affects cash flow.
- **Opportunity:** Increase volume sales of Office Supplies through bulk discounts to leverage cost efficiencies.

### Action

- Tier suppliers based on category profitability.
- Give priority contracts to technology suppliers to ensure supply chain resilience and margin protection.
- Monitor Furniture inventory closely to avoid excess stock and optimize capital usage.
- Implement volume discount strategies for Office Supplies to boost turnover and reduce overhead.


### 4. Profit Margin by Product Category

In [19]:
query = """
SELECT 
    Category,
    ROUND(SUM(Profit), 2) AS total_profit,
    ROUND(SUM(Sales), 2) AS total_sales,
    ROUND(SUM(Profit) * 100.0 / SUM(Sales), 2) AS profit_margin_pct
FROM orders
GROUP BY Category
ORDER BY profit_margin_pct DESC;
"""

pd.read_sql_query(query, conn)


,Category,total_profit,total_sales,profit_margin_pct
0,Technology,145438.50,836088.11,17.40
1,Office Supplies,122500.44,719040.62,17.04
2,Furniture,18451.25,741999.98,2.49


**Supply Chain Priority Matrix:**  

1. **Tech & Office Supplies** (17%+ margins)  
   - *Strategy:* Protect margins with **just-in-time inventory** and premium supplier partnerships  
   - *Risk:* Component shortages hurt high-value categories most  

2. **Furniture** (2.5% margin)  
   - *Red Flag:* Likely losing money after logistics costs  
   - *Action:* Renegotiate supplier contracts or exit unprofitable SKUs  

**Profit Levers:**  
- Shift warehouse space from Furniture to high-margin categories  
- Audit Furniture supply chain for hidden costs (damage/returns)

### 5. Sales Performance by Region and Segment

In [21]:
query = """
SELECT 
    Region,
    Segment,
    ROUND(SUM(Sales), 2) AS total_sales,
    ROUND(SUM(Profit), 2) AS total_profit
FROM orders
GROUP BY Region, Segment
ORDER BY total_profit DESC;
"""

pd.read_sql_query(query, conn)


,Region,Segment,total_sales,total_profit
0,West,Consumer,362880.82,57450.69
1,East,Consumer,350908.17,41190.96
2,West,Corporate,225855.25,34437.55
3,South,Consumer,195580.92,26913.63
4,East,Home Office,127463.74,26709.28
5,East,Corporate,200343.46,23605.44
6,Central,Corporate,157989.42,18713.46
7,West,Home Office,136721.86,16530.55
8,South,Corporate,121885.95,15215.40
9,Central,Home Office,91212.66,12438.50


### Supply Chain Optimization Insights

#### 1. Regional Profit Hotspots
- **West Region** leads with highest profit (**$57.4K from Consumers**), reflecting strong demand and efficient logistics.
- **Central Consumer segment** underperforms (**$8.5K profit** despite high sales), indicating possible pricing or cost inefficiencies that need addressing.

#### 2. Segment Prioritization
- **Consumer Segment** drives maximum profit (top 4 profit spots); prioritize stock availability and ensure fast replenishment cycles.
- **Home Office segment struggles in South** region with low profit (**$4.6K**); evaluate supplier costs, potential markdowns, and supply chain inefficiencies.

#### 3. Corporate Segment Consistency
- Shows mid-range profitability across regions; opportunity to optimize through bulk procurement and strengthened vendor contracts.

### Action Plan
- Boost supply chain agility in **West and East regions**, which are high-margin zones.
- Conduct a thorough audit of **Central Consumer segment costs** to identify and eliminate inefficiencies.
- Reassess supplier terms and logistics for **South Home Office** to improve profitability and reduce markdown risks.


### 6. Sales Velocity by Product Category

In [32]:
query = """
SELECT 
    Category,
    ROUND(SUM(Quantity) * 1.0 / COUNT(DISTINCT "Order Date"), 2) AS sales_velocity
FROM orders
WHERE "Order Date" IS NOT NULL
GROUP BY Category
ORDER BY sales_velocity DESC;
"""

pd.read_sql_query(query, conn)


,Category,sales_velocity
0,Office Supplies,19.95
1,Furniture,9.03
2,Technology,8.42


### Supply Chain Velocity Insights

#### 1. Office Supplies – Fast-Moving Essentials
- Highest sales velocity (**19.95 units/day**) indicates a need for frequent replenishment.
- Reliable suppliers are critical to avoid costly stockouts.
- **Action:** Maintain buffer stock levels and negotiate bulk discounts with vendors to balance availability and cost.

#### 2. Furniture & Technology – Slower but High-Value
- Lower turnover rates (**Furniture: 9.03 units/day, Technology: 8.42 units/day**) but higher profit margins (from prior insights).
- Requires optimized warehouse management focusing on bulk storage rather than rapid restocking.
- **Action:** Prioritize efficient use of warehouse space, ensuring premium storage conditions to prevent obsolescence and protect value.

### Strategic Takeaway
- Align inventory strategy with sales velocity for maximum supply chain efficiency:
  - **Office Supplies:** Adopt lean inventory with high-frequency restocking to meet fast demand.
  - **Technology and Furniture:** Leverage bulk shipments and premium storage solutions to safeguard high-value inventory and reduce handling costs.


### 7. Profitability per Product Unit Sold

In [31]:
query = """
SELECT 
    Category,
    ROUND(SUM(Profit) / SUM(Quantity), 2) AS profit_per_unit
FROM orders
WHERE Quantity > 0
GROUP BY Category
ORDER BY profit_per_unit DESC;
"""


pd.read_sql_query(query, conn)


,Category,profit_per_unit
0,Technology,20.96
1,Office Supplies,5.35
2,Furniture,2.30


### Supply Chain Profitability Insights

1. **Technology is the Profit Powerhouse**
   - Delivers **4x higher profit per unit ($20.96)** compared to Office Supplies.
   - **Priority:** Secure reliable suppliers and protect margins through strategic sourcing and supply chain resilience.

2. **Office Supplies - Volume Game**
   - Moderate profit of **$5.35 per unit** requires efficient, high-volume operations.
   - **Focus:** Streamline procurement, minimize handling costs, and leverage bulk ordering discounts.

3. **Furniture Margin Warning**
   - Low profit of **$2.30 per unit** highlights risks:
     - High logistics and warehousing costs.
     - Potential need for supplier renegotiation.
     - Consider product line rationalization or alternative distribution strategies.

### Action Plan

- **Technology:** Lock in long-term supplier contracts to ensure stable supply and margin protection.
- **Office Supplies:** Optimize bulk ordering and procurement processes to maximize volume efficiencies and reduce overhead.
- **Furniture:** Conduct a comprehensive cost-to-serve analysis to identify cost savings, supplier contract improvements, and potential product portfolio adjustments.


### 8. Return Rate by Product Category

In [24]:
query = """
SELECT 
    Category,
    COUNT(CASE WHEN Returned = 'Yes' THEN 1 END) * 1.0 / COUNT(*) AS return_rate
FROM orders
GROUP BY Category
ORDER BY return_rate DESC;
"""

pd.read_sql_query(query, conn)

,Category,return_rate
0,Technology,0.084507
1,Furniture,0.080622
2,Office Supplies,0.078506


### Supply Chain Risk Analysis

1. **Technology Has Highest Return Rate (8.45%)**
   - Despite being the most profitable category, technology products experience quality or expectation mismatches.
   - **Action:** Implement stricter quality control (QC) checks and enhance product descriptions to set accurate customer expectations.
   - **Mitigation:** Add pre-shipment testing for high-value tech items to reduce defects reaching customers.

2. **Furniture Close Second (8.06%)**
   - Returns likely stem from shipping damage or assembly difficulties.
   - **Solution:** Enhance packaging standards to better protect products during transit.
   - Provide assembly videos or detailed instructions to reduce user errors and returns.
   - **Mitigation:** Partner with specialized logistics providers experienced in handling fragile or complex items.

3. **Office Supplies Most Reliable (7.85%)**
   - Lowest return rate complements its fast sales velocity and operational efficiency.
   - **Opportunity:** Extend similar quality assurance and control practices to Technology and Furniture to reduce their return rates.

### Mitigation Strategy

- **Technology:** Introduce stringent pre-shipment inspections/testing, improve documentation and customer communication.
- **Furniture:** Upgrade packaging, reinforce shipping partnerships, and offer assembly support materials.
- **All Categories:** Perform detailed analysis of return reasons to detect any systemic product or process weaknesses and proactively address them.


### 9. Gross Margin Return on Investment (GMROI)

In [25]:
query = """
SELECT 
    Category,
    ROUND(SUM(Profit) / NULLIF(SUM(Sales - Profit), 0), 2) AS gmroi
FROM orders
GROUP BY Category
ORDER BY gmroi DESC;
"""

pd.read_sql_query(query, conn)


,Category,gmroi
0,Technology,0.21
1,Office Supplies,0.21
2,Furniture,0.03


High GMROI means you're earning more profit per inventory dollar. It’s useful in evaluating which categories are efficient to stock.

### 10. Average Discount by Product Category

In [30]:
query = """
SELECT 
    Category,
    ROUND(AVG(Discount), 2) AS avg_discount
FROM orders
GROUP BY Category
ORDER BY avg_discount DESC;
"""


pd.read_sql_query(query, conn)


,Category,avg_discount
0,Furniture,0.17
1,Office Supplies,0.16
2,Technology,0.13


- **Furniture's High Discounts (17%)** suggest overstock or low demand - optimize inventory and promotions  
- **Tech's Lower Discounts (13%)** reflect stronger pricing power - maintain premium positioning  
- **Office Supplies' Mid-Level Discounts (16%)** indicate competitive market - focus on volume efficiency

### 11. Return Rate by Region

In [29]:
query = """
SELECT 
    Region,
    COUNT(CASE WHEN Returned = 'Yes' THEN 1 END) * 1.0 / COUNT(*) AS return_rate
FROM orders
GROUP BY Region
ORDER BY return_rate DESC;
"""


pd.read_sql_query(query, conn)


,Region,return_rate
0,West,0.152982
1,East,0.052336
2,South,0.042593
3,Central,0.039621


- **West Region's Alarmingly High Returns (15.3%)** likely indicate logistics/shipping issues or stricter customer expectations - urgent investigation needed.  
- **East vs South/Central Discrepancy** (5.2% vs ~4%) suggests regional operational differences in fulfillment quality.  
- **Action Priority:** Benchmark West's return reasons against Central's best practices to identify correctable gaps.

Helps identify if certain regions are more prone to customer dissatisfaction or reverse logistics

### 12. Discount Effectiveness

In [35]:
query = """
SELECT 
    Category,
    ROUND(SUM(Profit) / NULLIF(SUM(Discount), 0), 2) AS profit_per_discount_dollar
FROM orders
WHERE Discount > 0
GROUP BY Category
ORDER BY profit_per_discount_dollar DESC;
"""


pd.read_sql_query(query, conn)


,Category,profit_per_discount_dollar
0,Technology,53.63
1,Office Supplies,-8.45
2,Furniture,-107.57


### 13. Profit Margin by Region

In [37]:
query = """
SELECT 
    Region,
    ROUND(SUM(Profit) * 100.0 / SUM(Sales), 2) AS profit_margin_pct
FROM orders
GROUP BY Region
ORDER BY profit_margin_pct DESC;
"""


pd.read_sql_query(query, conn)


,Region,profit_margin_pct
0,West,14.94
1,East,13.48
2,South,11.93
3,Central,7.92


### Regional Profitability Insights & Supply Chain Actions

#### 1. West Region Excellence (14.9% Margin)
- **Why Leading:**  
  Likely benefits from efficient coastal logistics, proximity to major ports, and a tech-savvy consumer base driving high-value sales and streamlined operations.
- **Action:**  
  Identify and replicate West Region's successful logistics practices, supplier relationships, and technology adoption in other regions to uplift overall performance.

#### 2. Central Region Underperformance (7.9% Margin)
- **Critical Gap:**  
  Margin is nearly half of the West Region’s, signaling significant profitability challenges.
- **Root Causes:**  
  - Higher inland transportation costs due to longer haul distances and less direct routes.  
  - Warehouse inefficiencies possibly related to layout, labor productivity, or automation use.  
  - Less favorable product mix skewing profitability downward.
- **Action:**  
  Conduct a thorough deep-dive into the Central Region’s cost structure:  
  - Analyze transportation modes, routes, and cost drivers.  
  - Evaluate facility locations and assess automation potential to improve throughput and reduce labor costs.  
  - Review product mix and consider strategic realignment if needed.

#### 3. East & South Transition Zones (13.5% & 11.9% Margins)
- **Opportunity:**  
  These regions have solid margins but still lag West Region performance, indicating room for improvement.
- **Improvement Levers:**  
  - Optimize distribution networks for reduced transportation time and cost.  
  - Regionalize inventory allocation based on demand patterns to improve service levels and reduce safety stock.

### Strategic Recommendations

- ✓ **Conduct Central Region Deep-Dive:**  
  - Develop detailed cost analyses focusing on transportation and warehousing.  
  - Identify opportunities for automation and facility optimization.

- ✓ **Develop Margin Improvement Playbook:**  
  - Extract best practices from the West Region’s supply chain, logistics, and supplier management.  
  - Facilitate cross-regional knowledge sharing and joint supplier negotiations to leverage scale and pricing.

- ✓ **Implement Regional Performance Dashboards:**  
  - Deploy weekly tracking of margin drivers, including transportation costs, inventory turnover, and service KPIs.  
  - Set up alert systems for rapid response to margin erosion or operational bottlenecks.


### 14. Top Returning Cities

In [38]:
query = """
SELECT 
    City,
    COUNT(*) AS total_returns
FROM orders
WHERE Returned = 'Yes'
GROUP BY City
ORDER BY total_returns DESC
LIMIT 10;
"""


pd.read_sql_query(query, conn)


,City,total_returns
0,Los Angeles,117
1,San Francisco,64
2,Seattle,60
3,New York City,45
4,San Diego,18
5,Columbus,17
6,Phoenix,16
7,Jackson,16
8,Salem,14
9,Philadelphia,13


### Supply Chain & Logistics Hotspot Analysis

#### 1. Critical Return Zones
- **Los Angeles dominates** with 117 returns (twice the runner-up San Francisco), indicating possible issues such as:  
  - Last-mile delivery challenges  
  - Higher customer expectations in metro areas  
  - Potential regional quality control failures  

#### 2. West Coast Concentration
- Top 3 cities (**Los Angeles, San Francisco, Seattle**) account for **53%** of the top 10 return locations.  
- **Action:** Conduct thorough audits of West Coast distribution centers and 3PL partners to identify and address root causes.

#### 3. Operational Red Flags
- Presence of smaller cities like **Jackson** and **Salem** in the top 10 returns points to:  
  - Systemic packaging or shipping issues  
  - Inconsistent return policies or handling across locations  

### Immediate Actions
- ✓ Perform root cause analysis segmented by city clusters:  
  - Metro vs non-metro return patterns  
  - Carrier performance and reliability comparisons  

- ✓ Implement targeted regional return prevention measures:  
  - Enhanced packaging standards specifically for West Coast shipments  
  - Pre-delivery quality control checks focusing on high-return cities  


### 15. Profit Contribution by Sub-Category

In [39]:
query = """
SELECT 
    "Sub-Category",
    ROUND(SUM(Profit) * 100.0 / (SELECT SUM(Profit) FROM orders), 2) AS profit_contribution_pct
FROM orders
GROUP BY "Sub-Category"
ORDER BY profit_contribution_pct DESC;
"""


pd.read_sql_query(query, conn)


,Sub-Category,profit_contribution_pct
0,Copiers,19.42
1,Phones,15.54
2,Accessories,14.64
3,Paper,11.89
4,Binders,10.56
5,Chairs,9.28
6,Storage,7.43
7,Appliances,6.33
8,Furnishings,4.56
9,Envelopes,2.43


### Supply Chain Profit Concentration Analysis

1. **Profit Power Zones (Top 20% Contributors = 60% Profit)**
   - **Copiers (19.4%), Phones (15.5%), Accessories (14.6%)** collectively generate nearly half of all profit.
   - **Action:** 
     - Prioritize inventory allocation to ensure availability of these high-profit sub-categories.
     - Strengthen supplier relationships and consider strategic partnerships or volume contracts.
     - Allocate resources for demand forecasting and supply chain resilience focused on these items.

2. **Hidden Opportunities**
   - **Paper (11.9%) and Binders (10.6%)** show strong profit density despite being less prominent.
   - **Opportunity:** Leverage their stable, predictable demand as reliable cash flow contributors.
   - Consider targeted promotions or bundling strategies to maximize sales with minimal risk.

3. **Profit Drains Requiring Intervention**
   - **Tables (-6.2%) and Bookcases (-1.2%)** are negatively impacting overall profitability.
   - **Urgent Action:**
     - Conduct immediate cost restructuring focusing on procurement, logistics, and production costs.
     - Assess product viability and consider product line discontinuation or replacement if profitability does not improve.
     - Investigate root causes of losses, including potential return rates, damage, or obsolescence.

4. **Marginal Performers**
   - Sub-categories from **Appliances (6.3%) down to Labels (1.9%)** exhibit thin margins needing operational efficiencies.
   - **Strategy:** 
     - Benchmark key processes, costs, and supplier performance against top performers like Copiers and Phones.
     - Identify best practices in inventory turnover, sourcing, and logistics.
     - Implement continuous improvement programs to enhance profitability.